In [ ]:
import ctypes
import time

SendInput = ctypes.windll.user32.SendInput


right_pressed=0x4D


left_pressed=0x4B

 
PUL = ctypes.POINTER(ctypes.c_ulong)
class KeyBdInput(ctypes.Structure):
    _fields_ = [("wVk", ctypes.c_ushort),
                ("wScan", ctypes.c_ushort),
                ("dwFlags", ctypes.c_ulong),
                ("time", ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class HardwareInput(ctypes.Structure):
    _fields_ = [("uMsg", ctypes.c_ulong),
                ("wParamL", ctypes.c_short),
                ("wParamH", ctypes.c_ushort)]

class MouseInput(ctypes.Structure):
    _fields_ = [("dx", ctypes.c_long),
                ("dy", ctypes.c_long),
                ("mouseData", ctypes.c_ulong),
                ("dwFlags", ctypes.c_ulong),
                ("time",ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class Input_I(ctypes.Union):
    _fields_ = [("ki", KeyBdInput),
                 ("mi", MouseInput),
                 ("hi", HardwareInput)]

class Input(ctypes.Structure):
    _fields_ = [("type", ctypes.c_ulong),
                ("ii", Input_I)]

def PressKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

def ReleaseKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008 | 0x0002, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

if __name__=='__main__':
    while (True):
        PressKey(0x11)
        time.sleep(1)
        ReleaseKey(0x11)
        time.sleep(1)

In [1]:

from imutils.video import VideoStream
import numpy as np
import cv2
import imutils
import time
from directkeys import right_pressed,left_pressed
from directkeys import PressKey, ReleaseKey 


break_key_pressed=left_pressed
accelerato_key_pressed=right_pressed


blueLower = np.array([110, 40, 40])
blueUpper = np.array([130,255,255])

vs = VideoStream(src=0).start()
 

time.sleep(2.0)
initial = True
flag = False
current_key_pressed = set()
circle_radius = 30
windowSize = 160
lr_counter = 0


break_pressed=False
accelerator_pressed=False
while True:
    keyPressed = False
    break_pressed=False
    accelerator_pressed=False
    
    frame = vs.read()
    height,width = frame.shape[:2]
 
   
    frame = cv2.flip(frame,1);
    
   
    frame = imutils.resize(frame, height=300)
    frame = imutils.resize(frame, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    
    mask = cv2.inRange(hsv, blueLower, blueUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
 
    
    left_mask = mask[:,0:width//2,]
    right_mask = mask[:,width//2:,]

   
    cnts_left = cv2.findContours(left_mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts_left = imutils.grab_contours(cnts_left)
    center_left = None

    cnts_right = cv2.findContours(right_mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts_right = imutils.grab_contours(cnts_right)
    center_right = None
    
    key_count=0
    key_pressed=0
    if len(cnts_left) > 0:
        
        c = max(cnts_left, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        
        center_left = (int(M["m10"] / (M["m00"]+0.000001)), int(M["m01"] / (M["m00"]+0.000001)))
       
        if radius > circle_radius:
            
            cv2.circle(frame, (int(x), int(y)), int(radius),
                (0, 0, 255), 2)
            cv2.circle(frame, center_left, 5, (0, 0, 255), -1)
            
            if center_left[1] > 250:
                cv2.putText(frame,'Break Applied',(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
                PressKey(break_key_pressed)
                break_pressed=True
                current_key_pressed.add(break_key_pressed)
               
                key_pressed=break_key_pressed
                keyPressed = True
                key_count=key_count+1
   
    if len(cnts_right) > 0:
        c2 = max(cnts_right, key=cv2.contourArea)
        ((x2, y2), radius2) = cv2.minEnclosingCircle(c2)
        M2 = cv2.moments(c2)
        center_right = (int(M2["m10"] / (M2["m00"]+0.000001)), int(M2["m01"] / (M2["m00"]+0.000001)))
        center_right = (center_right[0]+width//2,center_right[1])
        
        if radius2 > circle_radius:
            cv2.circle(frame, (int(x2)+width//2, int(y2)), int(radius2),
                (0, 255, 0), 2)
            cv2.circle(frame, center_right, 5, (0, 255, 0), -1)
            
            if center_right[1] >250 :
                cv2.putText(frame,'Acc. Applied',(350,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
                PressKey(accelerato_key_pressed)
                key_pressed=accelerato_key_pressed
                accelerator_pressed=True
                keyPressed = True
                current_key_pressed.add(accelerato_key_pressed)
                key_count=key_count+1
    
    frame_copy=frame.copy()
    
    frame_copy = cv2.rectangle(frame_copy,(0,height//2 ),(width//2,width),(255,255,255),1)
    cv2.putText(frame_copy,'Break',(10,280),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),3)
    
    frame_copy = cv2.rectangle(frame_copy,(width//2,height//2),(width,height),(255,255,255),1)
    cv2.putText(frame_copy,'Acceleration',(330,280),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),3)

    
    cv2.imshow("Frame", frame_copy)

   
    if not keyPressed and len(current_key_pressed) != 0:
        for key in current_key_pressed:
            ReleaseKey(key)
        current_key_pressed = set()
    elif key_count==1 and len(current_key_pressed)==2:    
            for key in current_key_pressed:             
                if key_pressed!=key:
                    ReleaseKey(key)
            current_key_pressed = set()
            for key in current_key_pressed:
                ReleaseKey(key)
            current_key_pressed = set()
        
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
 

vs.stop() 

cv2.destroyAllWindows()
